<a href="https://colab.research.google.com/github/KlavMar/recommandations_films/blob/main/project_2_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [157]:
import pandas as pd 
import numpy as np

import datetime as dt
from datetime import datetime 

import plotly.express as px
import os
import re

In [2]:
if input('Connect Drive ? ').upper()=='Y':
    from google.colab import drive
    drive.mount('/content/drive')
else: 
    pass
    


Connect Drive ? y
Mounted at /content/drive


# fonctions

In [3]:
#chemin="csv_project_2/"
chemin="/content/drive/MyDrive/Colab Notebooks/WILD_CODE_SCHOOL/PROJECT/PROJET 2/PROJET2_NEW/csv/"

In [7]:
def saveCsvOnfolder(df,cheminPath,nameFile):
        if os.path.exists(f'{cheminPath}'):
            df.to_csv(os.path.join(cheminPath,nameFile),sep=";")
        else:
            os.mkdir(f'{cheminPath}')
            df.to_csv(os.path.join(cheminPath,nameFile),sep=";")


def extractCsv(path,sepCsv,liste,name_liste):
    
    """ Warning : file only contains _ or camelCase no point in name file
        Onyl for csv simple no big file"""
    import os.path

    #import d'OS 
    #listDir = lecture du chemin 
    file_to_extract=os.listdir(path)
    #On itère sur le dossier pour exfilter l'ensemble des fichiers dans ce dossier
    count_liste=0 # permettra de retrouver le DF dans la liste
    try:
        for i in file_to_extract:
            df=pd.DataFrame()
            #On split le nom du fichier (qui ne doit pas contenir de point hors extension, et on prend la première sortie) 
            a=i.split('.')[0]
            try:
                #A chaque itération de boucle on ajoute à la liste le résultat de la lecture du fichier 
                chunk=pd.read_csv(f"{path}{i}",sep=sepCsv,chunksize=500000)
                for i in chunk:
                    df=pd.concat([df,i])
                
                liste.append(df)
                print(f'{a}={name_liste}[{count_liste}]')
                count_liste+=1
            except:
                #Si une erreur est détectée
                print(f"{i} n'est pas un fichier")
           
    except:
        pass
    return liste

In [5]:
def styleGraph(fig,title,nameAxis,nameOrdonnees):
    return fig.update_layout(
    title=title,
    xaxis_title=nameAxis,
    yaxis_title=nameOrdonnees,
    font=dict( 
        family="Arial",
        size=18,
        color="#000",
        
    ),
    )

def pxbar(df,x,y,title="",nameAxis="",nameOrdonnees="",color_graph="#33B4C3",color=None):
    # Création de la figure 
    fig=px.bar(data_frame=df,x=x,y=y,color_discrete_sequence=[color_graph],color=color)
    #application du thème 
    styleGraph(fig,title,nameAxis,nameOrdonnees)
    # return pour le plot offline
    return fig

def pxhisto(df,x,title,nameAxis,nameOrdonnees,color=None,color_graph="#33B4C3"):
    fig=px.histogram(data_frame=df,x=x,color=color,color_discrete_sequence=[color_graph])
    styleGraph(fig,title,nameAxis,nameOrdonnees)
    return fig

In [6]:
color_graph="#33B4C3"

# Détermination variable

In [8]:
%%time
liste_df=[]
extractCsv(chemin,";",liste_df,"liste_df")

df_title_akas=liste_df[0]
df_title_basics=liste_df[1]
df_title_crew=liste_df[2]
df_title_principal=liste_df[3]
df_rating=liste_df[4]
csv_filter n'est pas un fichier
non_exploit n'est pas un fichier
stat_total_bdd n'est pas un fichier
table_fr n'est pas un fichier


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/execution.py:1193: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/magics/execution.py:1193: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


df_name_basics=liste_df[5]
CPU times: user 3min 31s, sys: 17.7 s, total: 3min 49s
Wall time: 3min 58s


In [9]:
df_title_akas=liste_df[0]
df_title_basics=liste_df[1]
df_title_crew=liste_df[2]
df_title_principal=liste_df[3]
df_rating=liste_df[4]
df_name_basics=liste_df[5]

In [ ]:
liste_df=[]

# Exploitation

## Statistiques globale base de données

### df_title_basics

#### Gestion des dates

In [ ]:
df_title_basics.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,1894,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,1892,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,1892,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,1892,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,1893,1,"Comedy,Short"


In [ ]:
df_title_basics.replace('\\N',np.nan,inplace=True)
df_title_basics['startYear']=df_title_basics['startYear'].astype('float64')
df_title_basics['decade']=(df_title_basics['startYear'])//10*10
df_title_basics['startYear']=pd.to_datetime(df_title_basics['startYear'],format='%Y')
df_title_basics['startYear']=df_title_basics['startYear'].dt.year

#### Gestion des genres

In [ ]:
df_genres=pd.DataFrame()
for i in range(3):
    df_genres=pd.concat([df_genres,df_title_basics.genres.str.split(',',expand=True)[i]])

df_genres.rename(columns={0:'genres'},inplace=True)

In [ ]:
df_genres.value_counts().reset_index().rename(columns={0:'count'}).head()

,genres,count
0,Drama,2544088
1,Comedy,1850873
2,Talk-Show,1070377
3,Short,1014292
4,Documentary,857637


In [ ]:
pxbar(df_genres.value_counts(normalize=True).reset_index().rename(columns={0:'count'}).replace("\\N",np.nan).dropna(),x="genres",y="count",title="somme des genres",nameAxis="genres",nameOrdonnees="Nombre total",color="genres")

In [ ]:
#saveCsvOnfolder(df_genres.value_counts(normalize=True).reset_index().rename(columns={0:'count'}).replace("\\N",np.nan).dropna(),f'{chemin}/stat_total_bdd/','df_genres_all_db.csv')

In [ ]:
df_title_basics.columns

Index(['tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes',
       'genres', 'decade'],
      dtype='object')

#### Type de contenu

In [ ]:
pxbar(df_title_basics.titleType.value_counts(normalize=True).reset_index().rename(columns={'index':'type'}),x="type",y="titleType",title="percentage des types de contenu",nameAxis="Type contenu",nameOrdonnees="Percentage",color="type")

In [ ]:
#saveCsvOnfolder(df_title_basics.titleType.value_counts(normalize=True).reset_index().rename(columns={'index':'type_contenu'}),f'{chemin}/stat_total_bdd/','percentage_title_type_all_db.csv')

In [ ]:
contenu_per_decade=pd.DataFrame(df_title_basics.groupby(['decade'])['titleType'].value_counts(normalize=True)).rename(columns={'titleType':'count'}).reset_index()
contenu_per_decade.head()

,decade,titleType,count
0,1870.0,short,1.000000
1,1880.0,short,1.000000
2,1890.0,short,0.997197
3,1890.0,movie,0.002803
4,1900.0,short,0.992310


In [ ]:
fig=pxbar(contenu_per_decade,x='decade',y='count',color="titleType",title="Nb Contenu par Decade",nameAxis="decade",nameOrdonnees="pourcentage")
fig.update_xaxes(dtick=10)
fig.show()

In [ ]:
#saveCsvOnfolder(contenu_per_decade,f'{chemin}/stat_total_bdd/','total_contenu_par_decade.csv')

In [ ]:
# df_genres=[]
# df_title_basics=[]

### df_name_basics

In [63]:
df_name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0053137,tt0072308,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0071877,tt0037382,tt0117057"
2,nm0000003,Brigitte Bardot,1934,2014,"actress,soundtrack,music_department","tt0054452,tt0056404,tt0049189,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0080455,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0060827,tt0050976"


In [65]:
df_name_basics[df_name_basics.nconst=="nm9993701"]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
11804356,nm9993701,Sanjai Kuriakose,1925,2020,actor,tt8736744


In [67]:
df_title_basics[df_title_basics.tconst=="tt8736744"]

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres
8511966,tt8736744,short,Mirage The Last Men,2018,\N,"Drama,Short"


In [11]:
df_name_basics.replace("\\N",np.nan,inplace=True)

In [18]:
df_name_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11804374 entries, 0 to 11804373
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   deathYear          object
 4   primaryProfession  object
 5   knownForTitles     object
dtypes: object(6)
memory usage: 540.4+ MB


In [14]:
df_name_basics.isna().sum()

nconst                  0
primaryName             0
birthYear             649
deathYear            2202
primaryProfession      14
knownForTitles         12
dtype: int64

In [27]:
df_actor_actress=df_name_basics[(df_name_basics.primaryProfession=='actor') | (df_name_basics.primaryProfession=='actress')]
df_actor_actress

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
108,nm0000109,Yasmine Bleeth,1968,1998,actress,"tt0096542,tt0337851,tt0131857,tt0115285"
123,nm0000124,Jennifer Connelly,1970,1977,actress,"tt0102803,tt0180093,tt0268978,tt0315983"
142,nm0000143,Erika Eleniak,1969,1997,actress,"tt0105690,tt0106400,tt0083866,tt0094761"
156,nm0000157,Linda Hamilton,1956,1997,actress,"tt0088247,tt0103064,tt0118928,tt6450804"
265,nm0000266,Ursula Andress,1936,2006,actress,"tt0061452,tt0082186,tt0055928,tt0060177"
...,...,...,...,...,...,...
11804354,nm9993699,Dani Jacob,1925,2020,actor,tt8736744
11804355,nm9993700,Sexy Angel,1925,2020,actress,tt8736744
11804356,nm9993701,Sanjai Kuriakose,1925,2020,actor,tt8736744
11804357,nm9993702,Liz Matory,1925,2020,actor,tt8736744


In [25]:
df_actor_actress.isna().sum()

nconst                 0
primaryName            0
birthYear            208
deathYear            770
primaryProfession      0
knownForTitles         1
dtype: int64

In [66]:
df_nan_actor_actress=df_actor_actress[(df_actor_actress.deathYear.isna()) & (~df_actor_actress.birthYear.isna())].copy()
df_nan_actor_actress.birthYear=df_nan_actor_actress.birthYear.astype('int64')
df_nan_actor_actress['age']=2022-df_nan_actor_actress['birthYear']
df_nan_actor_actress

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,age
1000004,nm10083110,Andrei Morari,1956,NaN,actor,tt3333036,66
1000005,nm10083111,Amy Dentata,1956,NaN,actor,tt3333036,66
1000007,nm10083113,Iclal Nihal Cetin,1956,NaN,actor,"tt13303814,tt1985266,tt6127364,tt18646204",66
1000013,nm10083120,Su Sanad,1956,NaN,actress,"tt5988404,tt16453314,tt5175270",66
1000014,nm10083121,Sevgi Dalkiran,1956,NaN,actress,tt5988404,66
...,...,...,...,...,...,...,...
11500190,nm9638623,Ryôsei Tanaka,1928,NaN,actor,"tt15740902,tt8024758,tt19114232",94
11500191,nm9638624,Kô Hosokawa,1928,NaN,actor,"tt19114232,tt8820720,tt8024758,tt1706128",94
11500192,nm9638625,Yûki Kandagawa,1928,NaN,actress,"tt19114232,tt8820720,tt8024758,tt1706128",94
11500193,nm9638626,Mistral,1928,NaN,actress,tt0239180,94


In [45]:
df_not_nan_actor_actress=df_actor_actress[~(df_actor_actress.deathYear.isna()) & (~df_actor_actress.birthYear.isna())]
df_not_nan_actor_actress=df_not_nan_actor_actress.astype({'birthYear':'int64','deathYear':'int64'})
df_not_nan_actor_actress['age']=df_not_nan_actor_actress.deathYear-df_not_nan_actor_actress.birthYear
df_not_nan_actor_actress

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,age
108,nm0000109,Yasmine Bleeth,1968,1998,actress,"tt0096542,tt0337851,tt0131857,tt0115285",30
123,nm0000124,Jennifer Connelly,1970,1977,actress,"tt0102803,tt0180093,tt0268978,tt0315983",7
142,nm0000143,Erika Eleniak,1969,1997,actress,"tt0105690,tt0106400,tt0083866,tt0094761",28
156,nm0000157,Linda Hamilton,1956,1997,actress,"tt0088247,tt0103064,tt0118928,tt6450804",41
265,nm0000266,Ursula Andress,1936,2006,actress,"tt0061452,tt0082186,tt0055928,tt0060177",70
...,...,...,...,...,...,...,...
11804354,nm9993699,Dani Jacob,1925,2020,actor,tt8736744,95
11804355,nm9993700,Sexy Angel,1925,2020,actress,tt8736744,95
11804356,nm9993701,Sanjai Kuriakose,1925,2020,actor,tt8736744,95
11804357,nm9993702,Liz Matory,1925,2020,actor,tt8736744,95


In [75]:
df_actor_actress_age=pd.concat([df_nan_actor_actress,df_not_nan_actor_actress]).reset_index().sort_values(by="index").set_index('index')

### df_title_akas

In [ ]:
df_title_akas.dropna(inplace=True)

In [ ]:
df_region_akas_percentage=df_title_akas.region.value_counts(normalize=True).head(20)

In [ ]:
df_region_percentage=df_region_akas_percentage.copy()
#df_region_percentage.rename(columns={'region':'region','region':'count'},inplace=True)
df_region_percentage.replace("\\N",np.nan).dropna(inplace=True)
df_region_percentage=df_region_percentage.reset_index().replace("\\N",np.nan).rename(columns={'index':'langue'})
df_region_percentage
pxbar(df_region_percentage,x="langue",y="region",title="Représentation par langue",nameAxis="region",nameOrdonnees="Pourcentage")

In [ ]:
saveCsvOnfolder(df_region_percentage.head(10),f'{chemin}/stat_total_bdd/','df_region_percentage.csv')

In [ ]:
df_region_FR=df_title_akas[df_title_akas.region=="FR"]
df_region_FR.head()

,titleId,title,region
9,tt0000002,Le clown et ses chiens,FR
23,tt0000003,Pauvre Pierrot,FR
26,tt0000004,Un bon bock,FR
82,tt0000010,La sortie de l'usine Lumière,FR
84,tt0000010,La sortie des usines Lumière,FR


### df_title_crew

In [ ]:
df_title_crew.replace("\\N",np.nan,inplace=True)
df_title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN


In [ ]:
df_title_crew.directors.value_counts(normalize=True)

nm1203430     2.111484e-03
nm1409127     1.995647e-03
nm13220986    1.546559e-03
nm0022750     1.222175e-03
nm0565214     1.150089e-03
                  ...     
nm1603260     1.927416e-07
nm1082698     1.927416e-07
nm12704861    1.927416e-07
nm2219560     1.927416e-07
nm10538645    1.927416e-07
Name: directors, Length: 866325, dtype: float64

In [ ]:
df_title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN
...,...,...,...
9057830,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9057831,tt9916850,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
9057832,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9057833,tt9916856,nm10538645,nm6951431


### df_title_principal

In [ ]:
df_title_principal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51199889 entries, 0 to 51199888
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   tconst  object
 1   nconst  object
dtypes: object(2)
memory usage: 781.2+ MB


In [ ]:
df_title_principal.nconst.value_counts()

nm0914844     24095
nm0934807     16487
nm6352729     15883
nm0438506     14573
nm0438471     14308
              ...  
nm12630377        1
nm12631534        1
nm7162867         1
nm12867363        1
nm10538653        1
Name: nconst, Length: 4727914, dtype: int64

In [ ]:
df_title_principal

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271
...,...,...
51199884,tt9916880,nm10535738
51199885,tt9916880,nm0996406
51199886,tt9916880,nm1482639
51199887,tt9916880,nm2586970


#### Selection table => movies FR 

In [ ]:
df_table_fr=df_title_basics.merge(df_region_FR,how="inner",left_on="tconst",right_on="titleId").drop(columns=['titleId'])

In [ ]:
df_table_fr.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,decade,title,region
0,tt0000002,short,Le clown et ses chiens,1892.0,5,"Animation,Short",1890.0,Le clown et ses chiens,FR
1,tt0000003,short,Pauvre Pierrot,1892.0,4,"Animation,Comedy,Romance",1890.0,Pauvre Pierrot,FR
2,tt0000004,short,Un bon bock,1892.0,12,"Animation,Short",1890.0,Un bon bock,FR
3,tt0000010,short,Leaving the Factory,1895.0,1,"Documentary,Short",1890.0,La sortie de l'usine Lumière,FR
4,tt0000010,short,Leaving the Factory,1895.0,1,"Documentary,Short",1890.0,La sortie des usines Lumière,FR


In [ ]:
df_table_fr_type_percentage=df_table_fr.titleType.value_counts(normalize=True).reset_index().rename(columns={'index':'title','titleType':'percentage'})
pxbar(df_table_fr_type_percentage,x="title",y="percentage",title="",nameAxis="",nameOrdonnees="",color="title")

In [ ]:
df_table_fr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3877743 entries, 0 to 3877742
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   startYear       float64
 4   runtimeMinutes  object 
 5   genres          object 
 6   decade          float64
 7   title           object 
 8   region          object 
dtypes: float64(2), object(7)
memory usage: 295.8+ MB


In [ ]:
df_table_fr.decade.value_counts()

2010.0    1134936
2000.0     654605
1990.0     319488
2020.0     318084
1980.0     182871
1970.0     149761
1960.0      86894
1950.0      37200
1910.0      12660
1900.0       6770
1940.0       6342
1930.0       5668
1920.0       4555
1890.0       1189
1870.0          7
1880.0          1
Name: decade, dtype: int64

In [ ]:
df_movies_fr=df_table_fr[(df_table_fr.titleType=='movie')].dropna() #| (df_table_fr.titleType=='tvEpisode')]

In [ ]:
df_movies_genres=pd.DataFrame()
for i in range(3):
    df_movies_genres=pd.concat([df_movies_genres,df_movies_fr.genres.str.split(',',expand=True)[i]])

df_movies_genres.rename(columns={0:'genres'},inplace=True)

In [ ]:
df_movies_fr

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,decade,title,region
363,tt0000591,movie,The Prodigal Son,1907.0,90,Drama,1900.0,L'enfant prodigue,FR
459,tt0001790,movie,"Les Misérables, Part 1: Jean Valjean",1913.0,60,Drama,1910.0,Les misérables - Époque 1: Jean Valjean,FR
482,tt0002130,movie,Dante's Inferno,1911.0,71,"Adventure,Drama,Fantasy",1910.0,L'enfer,FR
493,tt0002199,movie,From the Manger to the Cross,1912.0,60,"Biography,Drama",1910.0,De la crèche à la croix,FR
516,tt0002423,movie,Passion,1919.0,85,"Biography,Drama,Romance",1910.0,Passion,FR
...,...,...,...,...,...,...,...,...,...
3877059,tt9913594,movie,Bacchanalia,2017.0,72,"Drama,Mystery,Thriller",2010.0,Bacchanalia,FR
3877151,tt9913936,movie,Paradise,2019.0,135,"Crime,Drama",2010.0,Le paradis de Diego,FR
3877236,tt9914164,movie,Une vie de château,2019.0,55,Documentary,2010.0,Une vie de château,FR
3877624,tt9916162,movie,Making of 'La Virgen de los sicarios',1999.0,45,Documentary,1990.0,Making of 'La Virgen de los sicarios',FR


In [ ]:
# df_table_fr_genres_split.genres.value_counts(normalize=True).reset_index()
# fig=pxbar(df_table_fr_genres_split.genres.value_counts(normalize=True).reset_index(),x="index",y="genres",title="Pourcentage des genres dans movies",nameAxis="genres",nameOrdonnees="Pourcentage")
# fig.show()

In [ ]:
saveCsvOnfolder(df_table_fr,f'{chemin}/table_fr','df_table_fr.csv')
saveCsvOnfolder(df_movies_genres,f'{chemin}/table_fr','df_movies_genres.csv')


In [ ]:
df_movies_fr['startYear']=df_movies_fr['startYear'].astype('int64')
df_movies_fr['decade']=df_movies_fr['decade'].astype('int64')
df_movies_fr.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,decade,title,region
363,tt0000591,movie,The Prodigal Son,1907,90,Drama,1900,L'enfant prodigue,FR
459,tt0001790,movie,"Les Misérables, Part 1: Jean Valjean",1913,60,Drama,1910,Les misérables - Époque 1: Jean Valjean,FR
482,tt0002130,movie,Dante's Inferno,1911,71,"Adventure,Drama,Fantasy",1910,L'enfer,FR
493,tt0002199,movie,From the Manger to the Cross,1912,60,"Biography,Drama",1910,De la crèche à la croix,FR
516,tt0002423,movie,Passion,1919,85,"Biography,Drama,Romance",1910,Passion,FR


In [ ]:
df_movies_dummies=df_movies_fr.merge(df_movies_fr.genres.str.get_dummies(sep=","),how="inner",left_index=True,right_index=True).drop(columns=['region','primaryTitle','titleType'])
df_movies_dummies.head()

,tconst,startYear,runtimeMinutes,genres,decade,title,Action,Adult,Adventure,Animation,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
363,tt0000591,1907,90,Drama,1900,L'enfant prodigue,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,tt0001790,1913,60,Drama,1910,Les misérables - Époque 1: Jean Valjean,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
482,tt0002130,1911,71,"Adventure,Drama,Fantasy",1910,L'enfer,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
493,tt0002199,1912,60,"Biography,Drama",1910,De la crèche à la croix,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516,tt0002423,1919,85,"Biography,Drama,Romance",1910,Passion,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_movies_dummies.insert(0,'title', df_movies_dummies.pop('title')) 
df_movies_dummies.insert(1,'genres', df_movies_dummies.pop('genres')) 
df_movies_dummies.head()

,title,genres,tconst,startYear,runtimeMinutes,decade,Action,Adult,Adventure,Animation,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
363,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,Les misérables - Époque 1: Jean Valjean,Drama,tt0001790,1913,60,1910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
482,L'enfer,"Adventure,Drama,Fantasy",tt0002130,1911,71,1910,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
493,De la crèche à la croix,"Biography,Drama",tt0002199,1912,60,1910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516,Passion,"Biography,Drama,Romance",tt0002423,1919,85,1910,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_count_decade_movies = df_movies_dummies.decade.value_counts(normalize=True).reset_index().sort_values(by="index",ascending=True)
fig=pxbar(df_count_decade_movies,x="index",y="decade",title="pourcentage de films par décade",nameAxis="decade",nameOrdonnees="pourcentage")
fig.update_xaxes(dtick=10)
fig.show()

In [ ]:
saveCsvOnfolder(df_movies_dummies,f'{chemin}/table_fr','df_movies_with_dummies.csv')

### Df_movies_dummies

/content/drive/MyDrive/Colab Notebooks/WILD_CODE_SCHOOL/PROJECT/PROJET 2/PROJET2_NEW/csv/table_fr/df_movies_with_dummies.csv

In [77]:
df_movies_dummies=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WILD_CODE_SCHOOL/PROJECT/PROJET 2/PROJET2_NEW/csv/table_fr/df_movies_with_dummies.csv',sep=";",index_col=0)

In [78]:
df_movies_dummies.reset_index().drop(columns='index',inplace=True)
df_movies_dummies

,title,genres,tconst,startYear,runtimeMinutes,decade,Action,Adult,Adventure,Animation,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
363,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,Les misérables - Époque 1: Jean Valjean,Drama,tt0001790,1913,60,1910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
482,L'enfer,"Adventure,Drama,Fantasy",tt0002130,1911,71,1910,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
493,De la crèche à la croix,"Biography,Drama",tt0002199,1912,60,1910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516,Passion,"Biography,Drama,Romance",tt0002423,1919,85,1910,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877059,Bacchanalia,"Drama,Mystery,Thriller",tt9913594,2017,72,2010,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3877151,Le paradis de Diego,"Crime,Drama",tt9913936,2019,135,2010,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3877236,Une vie de château,Documentary,tt9914164,2019,55,2010,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3877624,Making of 'La Virgen de los sicarios',Documentary,tt9916162,1999,45,1990,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_movies_dummies.runtimeMinutes=df_movies_dummies.runtimeMinutes.astype('int64')

In [ ]:
df_runtimes_movies = df_movies_dummies.runtimeMinutes.value_counts().reset_index()
df_runtimes_movies.describe(np.arange(0.5,1,0.05)).T

,count,mean,std,min,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
index,306.0,440.728758,2947.621488,2.0,177.5,192.75,208.0,226.25,241.5,260.75,285.0,320.25,374.5,483.75,43200.0
runtimeMinutes,306.0,250.369281,566.575445,1.0,7.5,14.50,22.0,44.00,114.0,184.25,286.0,486.50,950.0,1536.75,4910.0


In [79]:
df_movies_dummies_runtimes = df_movies_dummies[(df_movies_dummies.runtimeMinutes>=45) & (df_movies_dummies.runtimeMinutes<=490)]
df_movies_dummies_runtimes.head()

,title,genres,tconst,startYear,runtimeMinutes,decade,Action,Adult,Adventure,Animation,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
363,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,Les misérables - Époque 1: Jean Valjean,Drama,tt0001790,1913,60,1910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
482,L'enfer,"Adventure,Drama,Fantasy",tt0002130,1911,71,1910,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
493,De la crèche à la croix,"Biography,Drama",tt0002199,1912,60,1910,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516,Passion,"Biography,Drama,Romance",tt0002423,1919,85,1910,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
pd.DataFrame(df_movies_dummies_runtimes.startYear.describe(np.arange(0,1,0.1))).T

,count,mean,std,min,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
startYear,76554.0,1985.988897,28.737658,1905.0,1905.0,1941.0,1959.0,1971.0,1981.0,1992.0,2002.0,2009.0,2014.0,2018.0,2027.0


In [ ]:
fig=pxhisto(df_movies_dummies_runtimes,x="runtimeMinutes",title="count runtimes",nameAxis="runtimesMinutes",nameOrdonnees="nombre de films")
fig.show()

In [ ]:
fig= px.box(df_movies_dummies_runtimes,x="runtimeMinutes",color_discrete_sequence=[color_graph])
fig.show()

## Actors Actress

In [94]:
df_actors=df_name_basics.merge(df_title_basics,how="inner",left_on="nconst",right_on="tconst")


In [97]:
df_name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0053137,tt0072308,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0071877,tt0037382,tt0117057"
2,nm0000003,Brigitte Bardot,1934,2014,"actress,soundtrack,music_department","tt0054452,tt0056404,tt0049189,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0080455,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0060827,tt0050976"
...,...,...,...,...,...,...
11804369,nm9993714,Romeo del Rosario,1925,2020,"animation_department,art_department","tt14069590,tt2455546,tt11657662"
11804370,nm9993716,Essias Loberg,1925,2020,"animation_department,art_department","tt14069590,tt2455546,tt11657662"
11804371,nm9993717,Harikrishnan Rajan,1925,2020,cinematographer,tt8736744
11804372,nm9993718,Aayush Nair,1925,2020,cinematographer,tt8736744


In [98]:
df_title_basics

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,1894,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,1892,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,1892,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,1892,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,1893,1,"Comedy,Short"
...,...,...,...,...,...,...
9057830,tt9916848,tvEpisode,Episode #3.17,2010,\N,"Action,Drama,Family"
9057831,tt9916850,tvEpisode,Episode #3.19,2010,\N,"Action,Drama,Family"
9057832,tt9916852,tvEpisode,Episode #3.20,2010,\N,"Action,Drama,Family"
9057833,tt9916856,short,The Wind,2015,27,Short


In [99]:
df_title_principal

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271
...,...,...
51199884,tt9916880,nm10535738
51199885,tt9916880,nm0996406
51199886,tt9916880,nm1482639
51199887,tt9916880,nm2586970


In [114]:
df_movies_dummies_runtimes.drop_duplicates(subset='title',keep='first',inplace=True)
df_movies_dummies_runtimes.drop_duplicates(subset='tconst',keep='first',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [115]:
df_movies_dummies_runtimes.tconst.value_counts()

tt0000591     1
tt11093266    1
tt11080108    1
tt11080296    1
tt11080560    1
             ..
tt0089689     1
tt0089691     1
tt0089695     1
tt0089710     1
tt9916362     1
Name: tconst, Length: 64217, dtype: int64

In [116]:
df_movies=df_movies_dummies_runtimes.merge(df_title_principal,how="inner",left_on="tconst",right_on="tconst")

In [117]:
df_movies

,title,genres,tconst,startYear,runtimeMinutes,decade,Action,Adult,Adventure,Animation,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,nconst
0,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0906197
1,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0332182
2,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm1323543
3,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm1759558
4,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm0141150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592804,Les sorcières d'Akelarre,"Drama,History",tt9916362,2020,92,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm1893148
592805,Les sorcières d'Akelarre,"Drama,History",tt9916362,2020,92,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm3471432
592806,Les sorcières d'Akelarre,"Drama,History",tt9916362,2020,92,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm2970042
592807,Les sorcières d'Akelarre,"Drama,History",tt9916362,2020,92,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nm4065853


In [131]:
df_movies_with_act=df_movies.merge(df_name_basics,how="inner",left_on="nconst",right_on="nconst")

In [120]:
df_movies_with_act.head()

,title,genres,tconst,startYear,runtimeMinutes,decade,Action,Adult,Adventure,Animation,...,Talk-Show,Thriller,War,Western,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,nm0906197,Georges Wague,1874,1965,actor,"tt0432622,tt0764742,tt0007518,tt0434033"
1,La Vie de Christophe Colomb,History,tt0007518,1916,110,1910,0,0,0,0,...,0,0,0,0,nm0906197,Georges Wague,1874,1965,actor,"tt0432622,tt0764742,tt0007518,tt0434033"
2,L'enfant prodigue,Drama,tt0000591,1907,90,1900,0,0,0,0,...,0,0,0,0,nm0332182,Henri Gouget,1925,2016,actor,"tt3752844,tt0451296,tt0400955,tt0311681"
3,Le crime du Bouif,Crime,tt0156436,1922,73,1920,0,0,0,0,...,0,0,0,0,nm0332182,Henri Gouget,1925,2016,actor,"tt3752844,tt0451296,tt0400955,tt0311681"
4,Protéa,"Action,Thriller",tt0311681,1913,50,1910,1,0,0,0,...,0,1,0,0,nm0332182,Henri Gouget,1925,2016,actor,"tt3752844,tt0451296,tt0400955,tt0311681"


In [125]:
df_movies_with_act.columns

Index(['title', 'genres', 'tconst', 'startYear', 'runtimeMinutes', 'decade',
       'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'nconst', 'primaryName', 'birthYear',
       'deathYear', 'primaryProfession', 'knownForTitles'],
      dtype='object')

In [138]:
df_movies_filter=df_movies_with_act[(df_movies_with_act.Adult==0) & 
                                      ((df_movies_with_act.primaryProfession.str.contains('actor')) | 
                                       (df_movies_with_act.primaryProfession.str.contains('actress'))) & 
                                       (~df_movies_with_act.primaryProfession.str.contains('miscellaneous')) ]

In [139]:
df_movies_filter.primaryName.value_counts().head(50)

Gérard Depardieu          137
Alain Sarde               116
Michel Piccoli            110
Catherine Deneuve         108
Michel Audiard            107
Jesús Franco              107
John Wayne                103
Fernandel                 100
Jean-Claude Carrière       95
Marcello Mastroianni       95
Bernard Blier              95
Ben Hecht                  94
Michel Galabru             94
Frances Marion             92
Michel Serrault            92
Vittorio De Sica           91
Robert De Niro             89
Kuang Ni                   87
Jean-Louis Trintignant     87
Artur Brauner              87
Michael Caine              85
Michael Curtiz             83
Raoul Walsh                82
Jean Gabin                 82
Agenore Incrocci           81
James Mason                80
Nicolas Cage               80
Danielle Darrieux          80
Claude Brasseur            79
Charles Vanel              79
Christopher Lee            79
Jackie Chan                79
Isabelle Huppert           78
Fred MacMu

In [141]:
saveCsvOnfolder(df_movies_filter,f'{chemin}/table_fr/','df_movies_filter.csv')

In [148]:
df_table_actor=df_movies_filter.loc[:,['tconst','nconst','primaryName','birthYear','deathYear','primaryProfession','decade']]

In [159]:
df_table_actor['sexe']=df_table_actor.primaryProfession.apply(lambda x:'M' if re.findall('actor',x) else 'F')

In [168]:
pxbar(df_table_actor.sexe.value_counts(normalize=True).reset_index(),x="index",y="sexe",title="",nameAxis="",nameOrdonnees="")

In [169]:
df_table_actor.isna().sum()

tconst                0
nconst                0
primaryName           0
birthYear             4
deathYear            17
primaryProfession     0
decade                0
sexe                  0
dtype: int64

In [170]:
df_table_actor.deathYear.value_counts()

2020    7109
2021    6913
2016    6014
2014    5913
2017    5849
        ... 
1584       1
1842       1
1652       1
1912       1
1837       1
Name: deathYear, Length: 475, dtype: int64